# Post-Processing

(B): Method 1 is not supported and user should use method 2

This step in the workflow execution cycle is mostly to ensure that the correct type of return value is returned from the lattice. This step is quite resource consuming and since we'd like to keep the lattice as general as possible, for now it is run for every kind of workflow. But this can be avoided if in your workflow, you return an electron from the lattice instead of a custom object. Method (2) of writing a workflow is more preferrable than Method (1). Sometimes, certain return types will not even be supported, which is all the more reason to prefer "electron"izing wherever needed.

In [ ]:
import covalent as ct
import numpy as np
import random

# Method (1):
@ct.electron
def task_1(x):
    return x * 2

@ct.lattice
def workflow():
    res = task_1()
    res_list = random.sample(range(10, 30), res) # this will fail during graph construction time since `res` is still an Electron
    return np.array(res_list)



# Method (2):
@ct.electron
def task_2_new(x):
    res_list = random.sample(range(10, 30), x)
    return np.array(res_list)

@ct.lattice
def workflow_2():
    res_1 = task_1()
    return task_2_new(res_1)